<a href="https://colab.research.google.com/github/Snehlata1111/Elliptic-Curve-Optimization-using-PSO-GA-/blob/main/genetic_algo_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile fitness_function.py

import math
import os
def compute_fitness(a, b, p, n, weights):
    """
    Computes the fitness score of an elliptic curve based on security, attack resistance, and efficiency.

    Parameters:
        a, b (int): Elliptic curve coefficients.
        p (int): Prime modulus.
        n (int): Order of the curve.
        weights (tuple): (w1, w2, w3, w4) for security, Hasse validity, attack resistance, and efficiency.

    Returns:
        float: The fitness score.
    """
    w1, w2, w3, w4 = weights

    # Security Strength (S_n) - Based on bit length of order 'n'
    S_n = math.log2(n) / 256  # Normalized to [0,1] (256-bit max security)

    # Hasse Validity (H_s) - Always valid for well-formed curves
    H_s = 1.0

    # Attack Resistance (A_r) - Approximate value based on security strength
    A_r = (math.log2(n) / 256) * 0.8

    # Computational Efficiency (E_c) - Inversely proportional to field size
    E_c = 1 - (math.log2(p) / 521)  # Normalized to [0,1] (521-bit max field size)

    # Final fitness score
    fitness_score = (w1 * S_n) + (w2 * H_s) + (w3 * A_r) + (w4 * E_c)

    return fitness_score

Writing fitness_function.py


In [ ]:
%%writefile train_net2.py

import torch
import torch.nn as nn
import torch.optim as optim

# Loading trained Net1 model
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(4, 8)
        self.fc2 = nn.Linear(8, 4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Loading trained Net1 weights
net1 = Net1()
net1.load_state_dict(torch.load("/content/net1_weights.pth"))
net1.eval()

# Generate baseline weights from Net1
with torch.no_grad():
    baseline_weights = net1(torch.tensor([[0.49, 1.0, 0.39, 0.51]], dtype=torch.float32))

# ✅ Define Net2 (for system-specific tuning)
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(4, 8)
        self.fc2 = nn.Linear(8, 4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# ✅ System-specific training datasets
system_datasets = {
    "low_power": torch.tensor([[0.2, 1.0, 0.1, 0.7]], dtype=torch.float32),  # Optimized for efficiency
    "balanced": torch.tensor([[0.45, 1.0, 0.4, 0.45]], dtype=torch.float32),  # Balanced trade-offs
    "high_security": torch.tensor([[0.7, 1.0, 0.6, 0.2]], dtype=torch.float32)  # Prioritizes security
}

# ✅ Train Net2 for each system type
net2 = Net2()
criterion = nn.MSELoss()
optimizer = optim.Adam(net2.parameters(), lr=0.01)

epochs = 1000
for system, target_weights in system_datasets.items():
    print(f"Training Net2 for {system} system...")

    for epoch in range(epochs):
        optimizer.zero_grad()
        output = net2(baseline_weights)
        loss = criterion(output, target_weights)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}], System: {system}, Loss: {loss.item():.6f}")

    # ✅ Save the trained Net2 model
    model_filename = f"/content/net2_weights_{system}.pth"
    torch.save(net2.state_dict(), model_filename)

    print(f"Net2 training complete for {system}. Weights saved as {model_filename}\n")


Writing train_net2.py


In [ ]:
%%writefile genetic_algorithm.py

import torch
import random
import numpy as np
import torch.nn as nn
import os
from fitness_function import compute_fitness

# Define Genetic Algorithm Parameters
POPULATION_SIZE = 50
GENERATIONS = 100
MUTATION_RATE = 0.1
SELECTION_RATE = 0.5

# Define ranges for elliptic curve parameters
P_RANGE = (2**255, 2**256 - 1)
N_RANGE = (2**250, 2**256 - 1)
A_RANGE = (-5, 5)
B_RANGE = (1, 10**38)

# Load Net2 and get system-specific weights
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(4, 8)
        self.fc2 = nn.Linear(8, 4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

def load_net2_weights(system_type):
    weight_file = f"/content/net2_weights_{system_type}.pth"


    if not os.path.exists(weight_file):
        raise FileNotFoundError(f" Error: Net2 weights for {system_type} not found at {weight_file}. Train Net2 first.")

    net2 = Net2()  # Use the same Net2 architecture
    net2.load_state_dict(torch.load(weight_file, weights_only=True))
    net2.eval()

    with torch.no_grad():
        weights = net2(torch.tensor([[0.5, 1.0, 0.25, 0.2]], dtype=torch.float32))

    return weights.numpy()[0]  # Convert tensor to NumPy array
# Initialize a random elliptic curve
def random_curve():
    return {
        "a": random.uniform(*A_RANGE),
        "b": random.uniform(*B_RANGE),
        "p": random.randint(*P_RANGE),
        "n": random.randint(*N_RANGE)
    }

# Evaluate fitness for a population
def evaluate_population(population, weights):
    return sorted(population, key=lambda curve: compute_fitness(curve["a"], curve["b"], curve["p"], curve["n"], weights), reverse=True)

# Selection: Keep top % of population
def select_top_individuals(population):
    cutoff = int(len(population) * SELECTION_RATE)
    return population[:cutoff]

# Crossover: Generate new curves by mixing parent curves
def crossover(parent1, parent2):
    return {
        "a": (parent1["a"] + parent2["a"]) / 2,
        "b": (parent1["b"] + parent2["b"]) / 2,
        "p": random.choice([parent1["p"], parent2["p"]]),  # Maintain prime field structure
        "n": random.choice([parent1["n"], parent2["n"]])
    }

# Mutation: Introduce random changes to maintain diversity
def mutate(curve):
    if random.random() < MUTATION_RATE:
        curve["a"] += random.uniform(-0.5, 0.5)  # Slightly modify 'a'
    if random.random() < MUTATION_RATE:
        curve["b"] += random.uniform(-10**20, 10**20)  # Modify 'b'
    return curve

# Genetic Algorithm for Curve Optimization
def genetic_algorithm(system_type):
    print(f"\nRunning GA for {system_type.capitalize()} System...")
    weights = load_net2_weights(system_type)

    # Initialize population
    population = [random_curve() for _ in range(POPULATION_SIZE)]

    for generation in range(GENERATIONS):
        # Evaluate fitness
        population = evaluate_population(population, weights)

        # Select top individuals
        selected = select_top_individuals(population)

        # Generate new offspring via crossover
        new_population = selected[:]
        while len(new_population) < POPULATION_SIZE:
            parent1, parent2 = random.sample(selected, 2)
            offspring = mutate(crossover(parent1, parent2))
            new_population.append(offspring)

        population = new_population

        # Print best curve every 10 generations
        if generation % 10 == 0:
            best_curve = population[0]
            best_fitness = compute_fitness(best_curve["a"], best_curve["b"], best_curve["p"], best_curve["n"], weights)
            print(f"Gen {generation}: Best Fitness = {best_fitness:.6f}")

    # Return best curve found
    best_curve = population[0]
    print("\nFinal Best Curve:")
    print(best_curve)
    return best_curve

# Run GA for all system types
best_curves = {}
for system in ["low_power", "balanced", "high_security"]:
    best_curves[system] = genetic_algorithm(system)

# Save the best curves for future testing
import json
with open("/content/best_curves.json", "w") as f:
    json.dump(best_curves, f, indent=4)

print("\nBest curves saved in /content/best_curves.json")



Writing genetic_algorithm.py


In [ ]:
!python genetic_algorithm.py



Running GA for Low_power System...
Traceback (most recent call last):
  File "/content/genetic_algorithm.py", line 123, in <module>
    best_curves[system] = genetic_algorithm(system)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/genetic_algorithm.py", line 87, in genetic_algorithm
    weights = load_net2_weights(system_type)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/genetic_algorithm.py", line 39, in load_net2_weights
    raise FileNotFoundError(f" Error: Net2 weights for {system_type} not found at {weight_file}. Train Net2 first.")
FileNotFoundError:  Error: Net2 weights for low_power not found at /content/net2_weights_low_power.pth. Train Net2 first.


In [ ]:
import os

for system in ["low_power", "balanced", "high_security"]:
    file_path = f"/content/net2_weights_{system}.pth"
    print(f"{file_path}: {'Exists' if os.path.exists(file_path) else 'Missing'}")

/content/net2_weights_low_power.pth: Missing
/content/net2_weights_balanced.pth: Missing
/content/net2_weights_high_security.pth: Missing


In [ ]:
!python train_net2.py


Traceback (most recent call last):
  File "/content/train_net2.py", line 21, in <module>
    net1.load_state_dict(torch.load("/content/net1_weights.pth"))
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 1425, in load
    with _open_file_like(f, "rb") as opened_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 751, in _open_file_like
    return _open_file(name_or_buffer, mode)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/serialization.py", line 732, in __init__
    super().__init__(open(name, mode))
                     ^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/content/net1_weights.pth'
